In [1]:
from pyspark.sql import SparkSession, functions as func
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [2]:
spark = SparkSession.builder.appName("DiscoverHero").getOrCreate()

24/12/09 07:59:01 WARN Utils: Your hostname, Ngas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.115 instead (on interface en0)
24/12/09 07:59:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/09 07:59:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
schemas = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
])

In [15]:
names = spark.read.schema(schemas).option("sep", " ") .csv("../data/super_heroes/marvel_names")

In [16]:
lines = spark.read.text("../data/super_heroes/marvel_graph")

In [38]:
connections = (
    lines.withColumn("id", func.split(func.col("value"), " ")[0])
    .withColumn("connections", func.size(func.split(func.col("value"), " ")) - 1)
    .groupBy("id").agg(func.sum("connections").alias("connections"))
)
connections.show()


+----+-----------+
|  id|connections|
+----+-----------+
| 691|          7|
|1159|         12|
|3959|        143|
|1572|         36|
|2294|         15|
|1090|          5|
|3606|        172|
|3414|          8|
| 296|         18|
|4821|         17|
|2162|         42|
|1436|         10|
|1512|         12|
|6194|         15|
|6240|         12|
| 829|         38|
|2136|          7|
|5645|         21|
|2069|        264|
| 467|          1|
+----+-----------+
only showing top 20 rows



In [39]:
most_popular = connections.sort(func.col("connections").desc()).first()
most_popular

Row(id='859', connections=1937)

In [40]:
name = names.filter(func.col("id") == most_popular[0]).select("name").first()
name

Row(name='CAPTAIN AMERICA')